In [10]:
import pandas as pd
import numpy as np
# EJECUTA ESTO PRIMERO en tu notebook
%load_ext kedro.ipython

# Esto automáticamente configurará todo y creará:
# - context, session, catalog, pipelines
print("✅ Kedro extension loaded!")

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython
✅ Kedro extension loaded!


In [11]:
print("Datasets disponibles en el catálogo:")
catalog.keys()

Datasets disponibles en el catálogo:



[
    'animal_charity_donation_records',
    'synthetic_dog_breed_health_data',
    'cleaned_dog_breed',
    'cleaned_charity_donations',
    'cleaned_dog_breed_notebook',
    'cleaned_charity_donations_notebook',
    'parameters',
    'params:data_engineering',
    'params:data_engineering.imputation_strategy',
    'params:data_engineering.imputation_strategy.numerical',
    'params:data_engineering.imputation_strategy.categorical',
    'params:data_engineering.scaling',
    'params:data_engineering.scaling.standard_scaler',
    'params:data_engineering.scaling.robust_scaler',
    'params:data_engineering.outlier_limits',
    'params:data_engineering.outlier_limits.lower_percentile',
    'params:data_engineering.outlier_limits.upper_percentile',
    'params:data_engineering.outlier_limits.iqr_multiplier',
    'params:data_engineering.encode_columns',
    'params:donation_specific',
    'params:donation_specific.columns_to_drop',
    'params:donation_specific.amount_outlier_percentile

In [12]:

df_aac = catalog.load("aac_intakes_outcomes")




print(f" df_aac: {df_aac.shape}")

[09/14/25 16:25:34] INFO     Loading data from aac_intakes_outcomes (CSVDataset)...            ]8;id=311025;file://C:\Users\auror\Downloads\E1_ML_\Proyecto_ML_1\.venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=215486;file://C:\Users\auror\Downloads\E1_ML_\Proyecto_ML_1\.venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

 df_aac: (79672, 41)


In [13]:
#LIMPIEZA INICIAL
# Filtrar solo perros y eliminar duplicados
df_aac_clean = df_aac[df_aac['animal_type'] == 'Dog'].copy()
df_aac_clean = df_aac_clean.drop_duplicates(subset=['animal_id_intake'])

print(f"Registros después de limpieza: {df_aac_clean.shape[0]}")

Registros después de limpieza: 38909


In [14]:
#MANEJO DE VALORES NULOS

# Valores nulos por columna
null_counts = df_aac_clean.isnull().sum()
print("Valores nulos por columna:")
print(null_counts[null_counts > 0])

# Estrategia de imputación
df_aac_clean['intake_condition'] = df_aac_clean['intake_condition'].fillna('Unknown')
df_aac_clean['outcome_type'] = df_aac_clean['outcome_type'].fillna('Not Specified')
df_aac_clean['age_upon_intake_(years)'] = df_aac_clean['age_upon_intake_(years)'].fillna(
    df_aac_clean['age_upon_intake_(years)'].median()
)

Valores nulos por columna:
outcome_subtype     25631
outcome_type            1
sex_upon_outcome        1
sex_upon_intake         1
dtype: int64


In [15]:
# Salud al ingreso (binaria)
health_mapping = {
    'Normal': 1, 'Healthy': 1, 'Aged': 1,
    'Injured': 0, 'Sick': 0, 'Medical': 0,
    'Unknown': 0.5, 'Other': 0.5
}
df_aac_clean['health_status_intake'] = df_aac_clean['intake_condition'].map(health_mapping)

In [16]:
# Outcome favorable (binaria)
df_aac_clean['positive_outcome'] = df_aac_clean['outcome_type'].apply(
    lambda x: 1 if x in ['Adoption', 'Return to Owner', 'Transfer'] else 0
)

In [18]:
# Grupos etarios
age_bins = [0, 1, 3, 7, 10, 20]
age_labels = ['Cachorro', 'Joven', 'Adulto', 'Maduro', 'Senior']
df_aac_clean['age_group'] = pd.cut(
    df_aac_clean['age_upon_intake_(years)'], 
    bins=age_bins, 
    labels=age_labels
)

In [19]:
# Tiempo en refugio (semanas)
df_aac_clean['shelter_stay_weeks'] = df_aac_clean['time_in_shelter_days'] / 7

# Clasificación de estancia
df_aac_clean['stay_category'] = pd.cut(
    df_aac_clean['time_in_shelter_days'],
    bins=[0, 7, 30, 90, 365, float('inf')],
    labels=['Corta (<1 semana)', 'Media (1-4 semanas)', 'Larga (1-3 meses)', 
            'Muy larga (3-12 meses)', 'Crónica (>1 año)']
)

In [20]:
# Columnas relevantes para el análisis
selected_columns = [
    'animal_id_intake', 'breed', 'color', 'sex_upon_intake',
    'age_upon_intake_(years)', 'age_group', 'intake_condition',
    'health_status_intake', 'intake_type', 'outcome_type',
    'positive_outcome', 'time_in_shelter_days', 'shelter_stay_weeks',
    'stay_category', 'intake_datetime', 'outcome_datetime',
    'found_location'
]

df_aac_final = df_aac_clean[selected_columns].copy()

In [21]:
# Verificación final
print("=== CONTROL DE CALIDAD ===")
print(f"Registros finales: {df_aac_final.shape[0]}")
print(f"Valores nulos: {df_aac_final.isnull().sum().sum()}")
print(f"Distribución estado salud: {df_aac_final['health_status_intake'].value_counts()}")
print(f"Distribución outcomes positivos: {df_aac_final['positive_outcome'].value_counts()}")

=== CONTROL DE CALIDAD ===
Registros finales: 38909
Valores nulos: 789
Distribución estado salud: health_status_intake
1.0    35611
0.0     2616
0.5       86
Name: count, dtype: int64
Distribución outcomes positivos: positive_outcome
1    37131
0     1778
Name: count, dtype: int64


In [25]:
print("=== ESTADÍSTICAS DEL DATASET PREPARADO ===")
print(f"• Total perros: {len(df_aac_final)}")
print(f"• Edad promedio: {df_aac_final['age_upon_intake_(years)'].mean():.1f} años")
print(f"• Tiempo promedio en refugio: {df_aac_final['time_in_shelter_days'].mean():.1f} días")
print(f"• Porcentaje outcomes positivos: {df_aac_final['positive_outcome'].mean()*100:.1f}%")
print(f"• Distribución grupos etarios:")
print(df_aac_final['age_group'].value_counts())

=== ESTADÍSTICAS DEL DATASET PREPARADO ===
• Total perros: 38909
• Edad promedio: 2.7 años
• Tiempo promedio en refugio: 15.7 días
• Porcentaje outcomes positivos: 95.4%
• Distribución grupos etarios:
age_group
Cachorro    18738
Joven       10098
Adulto       6160
Maduro       2371
Senior       1351
Name: count, dtype: int64


In [27]:
# GUARDAR DATASET CORRECTAMENTE
import os
from pathlib import Path

# 1. Primero crear la carpeta si no existe
os.makedirs('data/02_intermediate', exist_ok=True)

# 2. Guardar el DataFrame CORRECTO (usa la variable que sí existe)
# En lugar de df_encoded, usa el DataFrame que realmente preparaste:
output_path = 'data/02_intermediate/cleaned_aac_intakes_outcomes_notebook.csv'
df_aac_final.to_csv(output_path, index=False)  # ← ¡CORREGIDO!

print(f"💾 Dataset guardado como: {output_path}")

# 3. Verificar que se creó
file_path = Path(output_path)
if file_path.exists():
    print(f"✅ Archivo verificado: {file_path} ({file_path.stat().st_size} bytes)")
else:
    print("❌ Error: El archivo no se creó")

💾 Dataset guardado como: data/02_intermediate/cleaned_aac_intakes_outcomes_notebook.csv
✅ Archivo verificado: data\02_intermediate\cleaned_aac_intakes_outcomes_notebook.csv (8827047 bytes)
